In [1]:
import pandas as pd


In [164]:
# Read all csv 
df_baseline = pd.read_csv('openml_baseline_scores.csv')
df_baseline_tabpfn = pd.read_csv('openml_baseline_tabpfn.csv')
df_modified_tabpfn = pd.read_csv('openml_modified_tabpfn.csv')


In [165]:


# drop index_x and index_y
df_baseline = df_baseline.drop(columns=['index_x', 'index_y'])

In [166]:
df_baseline.head()

,Name,# Features,# Categorical Features,# Instances,# Classes,# NaNs,Minority Class Size,id,classifier,roc,cross_entropy,accuracy,pred_time,train_time
0,pollen,6,1,3848,2,0,1924,871,lr,0.500000,0.693790,0.497,0.001502,0.001343
1,pollen,6,1,3848,2,0,1924,871,rf,0.518469,0.750826,0.521,0.018470,0.016239
2,pollen,6,1,3848,2,0,1924,871,svm,0.523551,0.693143,0.486,0.249156,0.217172
3,pollen,6,1,3848,2,0,1924,871,mlp,0.503956,0.694695,0.499,0.058430,0.055539
4,pollen,6,1,3848,2,0,1924,871,knn,0.501270,1.808627,0.498,0.047650,0.000294


In [167]:
# add classifier name to the dataframes
df_baseline_tabpfn['classifier'] = 'baseline_tabpfn'
df_modified_tabpfn['classifier'] = 'modified_tabpfn'

In [168]:
df_baseline_tabpfn.head()

,index,Name,# Features,# Categorical Features,# Instances,# Classes,# NaNs,Minority Class Size,id,roc,pred_time,cross_entropy,accuracy,classifier
0,0,pollen,6,1,3848,2,0,1924,871,0.491086,0.640528,0.694407,0.497,baseline_tabpfn
1,1,Sick_numeric,30,1,3772,2,0,231,41946,0.945509,0.997594,0.098889,0.973,baseline_tabpfn
2,2,jungle_chess_2pcs_endgame_rat_rat,47,27,3660,2,0,1605,41005,1.000000,0.166875,0.005538,1.000,baseline_tabpfn
3,4,led24,25,25,3200,10,0,296,40677,0.944567,0.993335,0.943728,0.700,baseline_tabpfn
4,5,led7,8,8,3200,10,0,270,40678,0.945100,0.504593,0.964591,0.720,baseline_tabpfn


In [169]:
# Move classifier column to the 9th column
cols = df_baseline_tabpfn.columns.tolist()
cols = cols[:9] + cols[-1:] + cols[9:-1]
df_baseline_tabpfn = df_baseline_tabpfn[cols]

In [170]:
# Move classifier column to the 9th column
cols = df_modified_tabpfn.columns.tolist()
cols = cols[:9] + cols[-1:] + cols[9:-1]
df_modified_tabpfn = df_modified_tabpfn[cols]

In [177]:
# Join all dataframes
df = pd.concat([df_baseline, df_baseline_tabpfn, df_modified_tabpfn])
df.head()

# sort by id
df = df.sort_values(by=['id'])

# combine train_time and pred_time
df['time'] = df['train_time'].fillna(0) + df['pred_time'].fillna(0)

In [184]:
# assign rank to each classifier based on roc
df['roc_rank'] = df.groupby(['id'])['roc'].rank(ascending=False)
df['accuracy_rank'] = df.groupby(['id'])['accuracy'].rank(ascending=False)
df['cross_entropy_rank'] = df.groupby(['id'])['cross_entropy'].rank(ascending=True)
df['time_rank'] = df.groupby(['id'])['time'].rank(ascending=False)

In [185]:
# sort by largest instances
df = df.sort_values(by=['# Instances','classifier'], ascending=False)


In [ ]:
# groupby id and classifier and report roc
df_grouped = df.groupby(['Name', 'classifier'])['roc'].mean().unstack()
df_grouped = df_grouped[['lr','knn','rf','svm', 'mlp', 'baseline_tabpfn', 'modified_tabpfn']]
df_grouped.head(20)

# round to 3 decimal places
df_grouped = df_grouped.round(3)

# Highlight the best classifier for each dataset
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

df_grouped.style.apply(highlight_max, axis=1)


In [176]:

# # convert classfier to columns
# df = df.pivot(index='id', columns='classifier', values=['roc', 'accuracy', 'cross_entropy', 'time', 'roc_rank', 'accuracy_rank', 'cross_entropy_rank', 'time_rank'])


# df.T.head()

In [186]:
# group by classifier and report the mean of the ranks
df_grouped = df.groupby(['classifier'])[["roc_rank", "accuracy_rank", "cross_entropy_rank", "time_rank"]].mean()


In [187]:

df_grouped = df_grouped.T

In [188]:
# Sort columns by classifier
df_grouped = df_grouped[['lr','knn','rf','svm', 'mlp', 'baseline_tabpfn', 'modified_tabpfn']]



In [189]:
df_grouped.T.reset_index()

,classifier,roc_rank,accuracy_rank,cross_entropy_rank,time_rank
0,lr,4.634241,4.348249,4.090535,5.622568
1,knn,5.175097,4.826848,5.785992,6.291829
2,rf,3.114786,3.346304,3.618677,2.801556
3,svm,3.661479,4.192607,3.809339,5.357977
4,mlp,3.221790,3.595331,3.905512,1.988327
5,baseline_tabpfn,3.463563,3.366397,2.795918,3.578947
6,modified_tabpfn,4.534413,4.109312,3.514170,2.072874


In [190]:
# highlight the maximum value in each column
df_grouped.style.highlight_min(axis=1)

classifier,lr,knn,rf,svm,mlp,baseline_tabpfn,modified_tabpfn
roc_rank,4.634241,5.175097,3.114786,3.661479,3.221790,3.463563,4.534413
accuracy_rank,4.348249,4.826848,3.346304,4.192607,3.595331,3.366397,4.109312
cross_entropy_rank,4.090535,5.785992,3.618677,3.809339,3.905512,2.795918,3.514170
time_rank,5.622568,6.291829,2.801556,5.357977,1.988327,3.578947,2.072874
